# CSJ 2024 — Constituency Summary
This notebook loads the Canada Summer Jobs data, filters to **2024 only**, and summarizes **total jobs created** and **total amount paid** per constituency.

All numbers are formatted with comma separators throughout (e.g. `14,258`).

## Cell 1 — Load, Clean & Aggregate
Read the raw CSV, filter to 2024, clean text fields, convert to numeric, and group by **Constituency** — summing total Amount Paid and Jobs Created. A **Cost Per Job** column is calculated.

In [ ]:
import pandas as pd
import numpy as np

# Load raw CSV
df = pd.read_csv('csj-results-master.csv', encoding='latin-1', dtype=str)
df.columns = ['Program Year','Region','Activity Constituency','Constituency','Employer','Amount Paid','Jobs Created']

# Filter to 2024 only
df = df[df['Program Year'].str.strip() == '2024'].copy()
df = df[['Constituency', 'Amount Paid', 'Jobs Created']]

# Clean text
df['Constituency'] = df['Constituency'].str.replace('\n', ' ', regex=False).str.replace(r'\s+', ' ', regex=True).str.strip()

# Convert to numeric
df['Amount Paid'] = pd.to_numeric(df['Amount Paid'].str.strip(), errors='coerce').fillna(0).astype(int)
df['Jobs Created'] = pd.to_numeric(df['Jobs Created'].str.strip(), errors='coerce').fillna(0).astype(int)

# Aggregate by constituency
summary = df.groupby('Constituency', as_index=False).agg(
    Total_Amount_Paid=('Amount Paid', 'sum'),
    Total_Jobs_Created=('Jobs Created', 'sum')
)
summary['Cost Per Job'] = np.where(
    summary['Total_Jobs_Created'] > 0,
    (summary['Total_Amount_Paid'] / summary['Total_Jobs_Created']).round(2),
    0.0
)
summary = summary.sort_values('Total_Amount_Paid', ascending=False).reset_index(drop=True)

print(f'2024 records: {len(df):,} employer entries')
print(f'Constituencies: {len(summary):,}')
print(f'Total Amount Paid: ${summary["Total_Amount_Paid"].sum():,}')
print(f'Total Jobs Created: {summary["Total_Jobs_Created"].sum():,}')
print()

# Display with comma formatting
display_df = summary.copy()
display_df['Total_Amount_Paid'] = display_df['Total_Amount_Paid'].apply(lambda x: f'{x:,}')
display_df['Total_Jobs_Created'] = display_df['Total_Jobs_Created'].apply(lambda x: f'{x:,}')
display_df['Cost Per Job'] = display_df['Cost Per Job'].apply(lambda x: f'{x:,.2f}')
display_df.head(10)

## Cell 2 — Highest & Lowest Cost Per Job
Identify the **10 constituencies with the highest** and **10 with the lowest** cost per job created, and visualise them in a horizontal bar chart.

In [ ]:
import matplotlib.pyplot as plt

# Sort by cost per job
sorted_df = summary.sort_values('Cost Per Job', ascending=False)
top_10 = sorted_df.head(10).copy()
bottom_10 = sorted_df.tail(10).copy()

# Print summaries with comma formatting
print('=== TOP 10 — HIGHEST Cost Per Job ===')
for _, row in top_10.iterrows():
    name = str(row['Constituency'])
    cpj = float(row['Cost Per Job'])
    jobs = int(row['Total_Jobs_Created'])
    paid = int(row['Total_Amount_Paid'])
    print(f'  {name:<45} ${cpj:>10,.2f}  ({jobs:,} jobs, ${paid:,} paid)')

print()
print('=== BOTTOM 10 — LOWEST Cost Per Job ===')
for _, row in bottom_10.iterrows():
    name = str(row['Constituency'])
    cpj = float(row['Cost Per Job'])
    jobs = int(row['Total_Jobs_Created'])
    paid = int(row['Total_Amount_Paid'])
    print(f'  {name:<45} ${cpj:>10,.2f}  ({jobs:,} jobs, ${paid:,} paid)')

# --- Horizontal bar chart ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
fig.suptitle('Cost Per Job by Constituency — 2024 CSJ Program',
             fontsize=15, fontweight='bold', y=1.02)

# Top 10 (highest) — BLUE
t10 = top_10.sort_values('Cost Per Job', ascending=True)
bars1 = ax1.barh(t10['Constituency'], t10['Cost Per Job'].astype(float),
                 color='#2980b9', edgecolor='white')
ax1.set_title('Top 10 — Highest Cost Per Job',
              fontsize=12, fontweight='bold', color='#1a5276')
ax1.set_xlabel('Cost Per Job ($)')
for bar in bars1:
    w = bar.get_width()
    ax1.text(w + 50, bar.get_y() + bar.get_height()/2,
             f'${w:,.0f}', va='center', fontsize=9, fontweight='bold')

# Bottom 10 (lowest) — GREEN
b10 = bottom_10.sort_values('Cost Per Job', ascending=True)
bars2 = ax2.barh(b10['Constituency'], b10['Cost Per Job'].astype(float),
                 color='#27ae60', edgecolor='white')
ax2.set_title('Bottom 10 — Lowest Cost Per Job',
              fontsize=12, fontweight='bold', color='#1e8449')
ax2.set_xlabel('Cost Per Job ($)')
for bar in bars2:
    w = bar.get_width()
    ax2.text(w + 20, bar.get_y() + bar.get_height()/2,
             f'${w:,.0f}', va='center', fontsize=9, fontweight='bold')

for ax in (ax1, ax2):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='y', labelsize=9)

plt.tight_layout()
plt.savefig('cost_per_job_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print('\nChart saved to cost_per_job_analysis.png')

## Cell 3 — Constituency Heatmap
A heatmap of **all 338 constituencies** colored by **Cost Per Job**.
- **Green** = most efficient (lowest cost per job — more jobs per dollar)
- **Blue** = least efficient (highest cost per job — fewer jobs per dollar)

Constituencies are sorted from most efficient (top-left) to least efficient (bottom-right).

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# Sort constituencies by cost per job (lowest = most efficient first)
heatmap_df = summary.sort_values('Cost Per Job', ascending=True).reset_index(drop=True)
n = len(heatmap_df)

# Build a grid: 18 columns so labels are readable
cols = 18
rows = int(np.ceil(n / cols))

# Pad data to fill the grid
values = heatmap_df['Cost Per Job'].values.astype(float)
names = heatmap_df['Constituency'].values
pad = rows * cols - n
values = np.append(values, [np.nan] * pad)
names = np.append(names, [''] * pad)

val_grid = values.reshape(rows, cols)
name_grid = names.reshape(rows, cols)

# Custom colormap: green (efficient) → yellow (mid) → blue (expensive)
cmap = mcolors.LinearSegmentedColormap.from_list(
    'efficiency', ['#27ae60', '#f1c40f', '#2980b9'], N=256
)
cmap.set_bad(color='#f5f5f5')  # empty cells

# Plot
fig, ax = plt.subplots(figsize=(28, 16))
im = ax.imshow(val_grid, cmap=cmap, aspect='auto')

# Add constituency names inside each cell
for i in range(rows):
    for j in range(cols):
        if name_grid[i, j] != '':
            # Shorten long names
            label = name_grid[i, j]
            if len(label) > 20:
                label = label[:18] + '..'
            val = val_grid[i, j]
            ax.text(j, i - 0.15, label, ha='center', va='center',
                    fontsize=5.5, fontweight='bold', color='white',
                    path_effects=[__import__('matplotlib.patheffects', fromlist=['withStroke']).withStroke(linewidth=2, foreground='black')])
            ax.text(j, i + 0.2, f'${val:,.0f}', ha='center', va='center',
                    fontsize=5, color='white',
                    path_effects=[__import__('matplotlib.patheffects', fromlist=['withStroke']).withStroke(linewidth=2, foreground='black')])

# Colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.6, pad=0.02)
cbar.set_label('Cost Per Job ($)', fontsize=12, fontweight='bold')
cbar.ax.tick_params(labelsize=10)

ax.set_title('All Constituencies — Cost Per Job Heatmap (2024 CSJ)\nGreen = Most Efficient  |  Blue = Least Efficient',
             fontsize=16, fontweight='bold', pad=20)
ax.set_xticks([])
ax.set_yticks([])
ax.spines[:].set_visible(False)

plt.tight_layout()
plt.savefig('constituency_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()
print(f'Heatmap saved to constituency_heatmap.png')
print(f'Grid: {rows} rows x {cols} cols = {n} constituencies')

## Cell 4 — Save Cleaned File
Export the constituency summary to `csj-results-cleaned.csv` (UTF-8) with comma-formatted numbers.

In [ ]:
import os

# Format for export
export = summary.copy()
export['Total_Amount_Paid'] = export['Total_Amount_Paid'].apply(lambda x: f'{int(x):,}')
export['Total_Jobs_Created'] = export['Total_Jobs_Created'].apply(lambda x: f'{int(x):,}')
export['Cost Per Job'] = export['Cost Per Job'].apply(lambda x: f'{float(x):,.2f}')

output_file = 'csj-results-cleaned.csv'
export.to_csv(output_file, index=False, encoding='utf-8')

size_kb = os.path.getsize(output_file) / 1024
print(f'Saved: {output_file}')
print(f'File size: {size_kb:.1f} KB')
print(f'Rows: {len(export):,} constituencies')
print(f'Columns: {list(export.columns)}')
print()
export